In [43]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-c6410a8bb202ca06.parquet', 'validation': 'data/validation-00000-of-00001-d21ad392180d1f79.parquet', 'test': 'data/test-00000-of-00001-d20b0e7149fa6eeb.parquet'}
df = pd.read_parquet("hf://datasets/bstee615/bigvul/" + splits["train"])

In [46]:
# df.info
vul_counts = df['vul'].value_counts()

vul_counts


vul
0    142194
1      8714
Name: count, dtype: int64

In [47]:
df.describe()

,vul
count,150908.000000
mean,0.057744
std,0.233259
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [18]:
df_filtered = df[df['CWE ID'].notna()]

vul_counts = df_filtered['vul'].value_counts()

vul_counts

vul
0    114937
1      6996
Name: count, dtype: int64

In [41]:
from sklearn.model_selection import train_test_split

# 分离 vul=0 和 vul=1 的数据
vul_0 = df_filtered[df_filtered['vul'] == 0]
vul_1 = df_filtered[df_filtered['vul'] == 1]

# 确保 vul=0 和 vul=1 的平衡，以少数类的数量为基准
minority_count = min(len(vul_0), len(vul_1))
vul_0_balanced = vul_0.sample(n=minority_count, random_state=42)
vul_1_balanced = vul_1.sample(n=minority_count, random_state=42)

# 合并平衡后的数据
balanced_df = pd.concat([vul_0_balanced, vul_1_balanced]).sample(frac=1, random_state=42)  # 打乱顺序

# 按 8:1:1 划分训练、验证、测试集
train_val_df, test_df = train_test_split(balanced_df, test_size=0.1, random_state=42, stratify=balanced_df['vul'])
train_df, val_df = train_test_split(train_val_df, test_size=0.1111, random_state=42, stratify=train_val_df['vul'])

# 输出各数据集大小
train_df.to_csv('data/train.csv', index=False)
val_df.to_csv('data/val.csv', index=False)
test_df.to_csv('data/test.csv', index=False)

{
    "训练集保存路径": "data/train.csv",
    "验证集保存路径": "data/val.csv",
    "测试集保存路径": "data/test.csv"
}

{'训练集保存路径': 'data/train.csv',
 '验证集保存路径': 'data/val.csv',
 '测试集保存路径': 'data/test.csv'}

In [4]:
train_df = pd.read_csv('data/train.csv')
val_df = pd.read_csv('data/val.csv')
test_df = pd.read_csv('data/test.csv')

In [7]:
unique_cwe_ids = df_filtered['CWE ID'].unique()

# 打印所有独特的 CWE ID
print(f"Unique CWE IDs ({len(unique_cwe_ids)} total):")
print(unique_cwe_ids)

Unique CWE IDs (90 total):
['CWE-119' 'CWE-732' 'CWE-189' 'CWE-415' 'CWE-287' 'CWE-20' 'CWE-200'
 'CWE-264' 'CWE-416' 'CWE-787' 'CWE-1021' 'CWE-254' 'CWE-125' 'CWE-284'
 'CWE-617' 'CWE-388' 'CWE-79' 'CWE-476' 'CWE-190' 'CWE-399' 'CWE-94'
 'CWE-404' 'CWE-772' 'CWE-89' 'CWE-362' 'CWE-400' 'CWE-59' 'CWE-310'
 'CWE-22' 'CWE-835' 'CWE-269' 'CWE-134' 'CWE-93' 'CWE-674' 'CWE-191'
 'CWE-281' 'CWE-285' 'CWE-704' 'CWE-834' 'CWE-17' 'CWE-436' 'CWE-693'
 'CWE-19' 'CWE-369' 'CWE-354' 'CWE-311' 'CWE-611' 'CWE-120' 'CWE-16'
 'CWE-754' 'CWE-74' 'CWE-909' 'CWE-78' 'CWE-361' 'CWE-668' 'CWE-77'
 'CWE-494' 'CWE-346' 'CWE-295' 'CWE-290' 'CWE-601' 'CWE-862' 'CWE-358'
 'CWE-706' 'CWE-532' 'CWE-330' 'CWE-172' 'CWE-320' 'CWE-18' 'CWE-129'
 'CWE-918' 'CWE-255' 'CWE-682' 'CWE-502' 'CWE-770' 'CWE-347' 'CWE-352'
 'CWE-426' 'CWE-345' 'CWE-755' 'CWE-90' 'CWE-665' 'CWE-763' 'CWE-522'
 'CWE-824' 'CWE-664' 'CWE-327' 'CWE-769' 'CWE-252' 'CWE-209']


In [8]:
unique_cwe_ids = set(train_df['CWE ID'].unique())

# 打印所有独特的 CWE ID
print(f"Unique CWE IDs ({len(unique_cwe_ids)} total):")
print(unique_cwe_ids)

all_cwe_ids = set(df_filtered['CWE ID'].unique())

non_overlapping_cwe_ids = all_cwe_ids - (unique_cwe_ids)


print(f"Non-overlapping CWE IDs ({len(non_overlapping_cwe_ids)} total):")
print(non_overlapping_cwe_ids)

Unique CWE IDs (83 total):
{'CWE-255', 'CWE-835', 'CWE-436', 'CWE-494', 'CWE-347', 'CWE-770', 'CWE-682', 'CWE-129', 'CWE-18', 'CWE-252', 'CWE-787', 'CWE-295', 'CWE-287', 'CWE-59', 'CWE-209', 'CWE-388', 'CWE-17', 'CWE-918', 'CWE-269', 'CWE-369', 'CWE-119', 'CWE-327', 'CWE-22', 'CWE-79', 'CWE-665', 'CWE-254', 'CWE-824', 'CWE-200', 'CWE-668', 'CWE-345', 'CWE-78', 'CWE-862', 'CWE-674', 'CWE-361', 'CWE-404', 'CWE-77', 'CWE-362', 'CWE-763', 'CWE-20', 'CWE-284', 'CWE-426', 'CWE-909', 'CWE-400', 'CWE-704', 'CWE-502', 'CWE-399', 'CWE-358', 'CWE-311', 'CWE-290', 'CWE-264', 'CWE-190', 'CWE-354', 'CWE-285', 'CWE-89', 'CWE-320', 'CWE-134', 'CWE-16', 'CWE-74', 'CWE-94', 'CWE-125', 'CWE-532', 'CWE-522', 'CWE-191', 'CWE-172', 'CWE-617', 'CWE-281', 'CWE-189', 'CWE-90', 'CWE-416', 'CWE-754', 'CWE-310', 'CWE-19', 'CWE-611', 'CWE-1021', 'CWE-330', 'CWE-834', 'CWE-476', 'CWE-601', 'CWE-772', 'CWE-415', 'CWE-346', 'CWE-120', 'CWE-732'}
Non-overlapping CWE IDs (7 total):
{'CWE-769', 'CWE-755', 'CWE-693', 'CW

In [10]:
vul_counts = train_df['vul'].value_counts()
print(vul_counts)

vul_counts = test_df['vul'].value_counts()
print(vul_counts)

vul_counts = val_df['vul'].value_counts()
print(vul_counts)



vul
1    5597
0    5596
Name: count, dtype: int64
vul
1    700
0    700
Name: count, dtype: int64
vul
0    700
1    699
Name: count, dtype: int64


In [34]:
import json

# Define the file path for GPT-4 fine-tuning data
gpt4_fine_tuning_file = "data/gpt4_fine_tuning_train.jsonl"

# Create GPT-4 fine-tuning data format (messages structure)
with open(gpt4_fine_tuning_file, 'w') as f:
    for _, row in train_df.iterrows():
        # Get the prompt (vulnerability code snippet)
        prompt = row['func_before']  # Replace with the actual column name containing code snippets
        
        # Construct the completion (vulnerability class and CWE information)
        vul_label = "vul 1" if row['vul'] == 1 else "vul 0"
        cwe = f"CWE: {row['CWE ID']}" if row['CWE ID'] else "CWE: None"
        completion = f"Vulnerability Class: {vul_label}\nCWE Number: {cwe}"
        
        # Format as messages for GPT-4
        messages = [
            {"role": "system", "content": "You are an assistant that detects vulnerabilities in code and provides their CWE number. Out put 1 and CWE number if the code is vulnerable, 0 otherwise."},
            {"role": "user", "content": prompt.strip()},
            {"role": "assistant", "content": completion.strip()}
        ]
        
        # Write to JSONL file
        json.dump({"messages": messages}, f)
        f.write('\n')

print(f"GPT-4 fine-tuning data has been saved to {gpt4_fine_tuning_file}")


GPT-4 fine-tuning data has been saved to data/gpt4_fine_tuning_train.jsonl


In [35]:
data_path = "data/gpt4_fine_tuning_train.jsonl"

with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

print("Num Examples:", len(dataset))
print("First Example:")
for message in dataset[0]:
    print(f"\t{message}: {dataset[0][message]}")

Num Examples: 11193
First Example:
	messages: [{'role': 'system', 'content': 'You are an assistant that detects vulnerabilities in code and provides their CWE number. Out put 1 and CWE number if the code is vulnerable, 0 otherwise.'}, {'role': 'user', 'content': 'static bool arcmsr_hbaB_get_config(struct AdapterControlBlock *acb)\n{\n\tstruct MessageUnit_B *reg = acb->pmuB;\n\tchar *acb_firm_model = acb->firm_model;\n\tchar *acb_firm_version = acb->firm_version;\n\tchar *acb_device_map = acb->device_map;\n\tchar __iomem *iop_firm_model;\n\t/*firm_model,15,60-67*/\n\tchar __iomem *iop_firm_version;\n\t/*firm_version,17,68-83*/\n\tchar __iomem *iop_device_map;\n\t/*firm_version,21,84-99*/\n\tint count;\n\n\tiop_firm_model = (char __iomem *)(&reg->message_rwbuffer[15]);\t/*firm_model,15,60-67*/\n\tiop_firm_version = (char __iomem *)(&reg->message_rwbuffer[17]);\t/*firm_version,17,68-83*/\n\tiop_device_map = (char __iomem *)(&reg->message_rwbuffer[21]);\t/*firm_version,21,84-99*/\n\n\tarcm

In [49]:
from collections import defaultdict

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["Examples not formatted as dictionaries"] += 1
        continue
    
    messages = ex.get("messages", None)
    
    if not messages:
        format_errors["Missing_msg_list"] += 1
        continue
    
    for message in messages:
        if 'role' not in message or 'content' not in message:
            format_errors["Missing_'message'_key"] += 1
    
        if any(k not in ('role', 'content', 'name', 'function_call') for k in message):
            format_errors["unrecognized_key"] += 1
        
        if message.get('role', None) not in ('system', 'user', 'assistant', 'function'):
            format_errors["unrecognized_role"] += 1
        
        content = message.get('content', None)
        function_call = message.get('function_call', None)
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
        
    if not any(message.get('role', None) == 'assistant' for message in messages):
        format_errors["example_missing_assistant_msg"] += 1

In [50]:
if format_errors:
    print("Format errors:")
    for error, count in format_errors.items():
        print(f"\t{error}: {count}")
else:
    print("No format errors found.")

No format errors found.


In [35]:
from sklearn.utils import resample

# 选择100条数据，保持 vul=0 和 vul=1 的平衡
num_samples_per_class = 500  # 平均分配

# 分离 vul=0 和 vul=1 的数据
vul_0_data = train_df[train_df['vul'] == 0]
vul_1_data = train_df[train_df['vul'] == 1]

# 随机采样 50 条 vul=0 和 50 条 vul=1 的数据
vul_0_sampled = resample(vul_0_data, replace=False, n_samples=num_samples_per_class, random_state=42)
vul_1_sampled = resample(vul_1_data, replace=False, n_samples=num_samples_per_class, random_state=42)

# 合并采样后的数据并打乱顺序
sampled_data = pd.concat([vul_0_sampled, vul_1_sampled]).sample(frac=1, random_state=42)

# 保存新的小规模训练集
sampled_data_file = "data/sample_train_data.csv"
sampled_data.to_csv(sampled_data_file, index=False)

print(f"Sampled training data saved to {sampled_data_file}, total {len(sampled_data)} samples:")
print(sampled_data['vul'].value_counts())


Sampled training data saved to data/sample_train_data.csv, total 1000 samples:
vul
1    500
0    500
Name: count, dtype: int64


In [36]:
import json

# Define the file path for GPT-4 fine-tuning data
gpt4_fine_tuning_file = "data/gpt4_fine_tuning_train_sample.jsonl"

# Create GPT-4 fine-tuning data format (messages structure)
with open(gpt4_fine_tuning_file, 'w') as f:
    for _, row in sampled_data.iterrows():
        # Get the prompt (vulnerability code snippet)
        prompt = row['func_before']  # Replace with the actual column name containing code snippets
        
        # Construct the completion (vulnerability class and CWE information)
        vul_label = "vul 1" if row['vul'] == 1 else "vul 0"
        cwe = f"CWE: {row['CWE ID']}" if row['CWE ID'] else "CWE: None"
        completion = f"Vulnerability Class: {vul_label}\nCWE Number: {cwe}"
        
        # Format as messages for GPT-4
        messages = [
            {"role": "system", "content": "You are an assistant that detects vulnerabilities in code and provides their CWE number. Out put 1 and CWE number if the code is vulnerable, 0 otherwise."},
            {"role": "user", "content": prompt.strip()},
            {"role": "assistant", "content": completion.strip()}
        ]
        
        # Write to JSONL file
        json.dump({"messages": messages}, f)
        f.write('\n')

print(f"GPT-4 fine-tuning data has been saved to {gpt4_fine_tuning_file}")

GPT-4 fine-tuning data has been saved to data/gpt4_fine_tuning_train_sample.jsonl


In [57]:
import openai
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()

openai.api_key = os.getenv("OPENAI_API_KEY2")

gpt4_sample_fine_tuning_file = "data/gpt4_fine_tuning_train_sample.jsonl"

if not openai.api_key:
    raise ValueError("API key not found. Please set OPENAI_API_KEY in your .env file.")

fine_tuning_file = "data/gpt4_fine_tuning_train_sample.jsonl"  # 替换为你的 JSONL 文件路径

file = client.files.create(
    file = open(fine_tuning_file, 'rb'),
    purpose = "fine-tune"
)

print(file)

ValueError: API key not found. Please set OPENAI_API_KEY in your .env file.

In [2]:
fine_tuning_job = client.fine_tuning.jobs.create(
    training_file = file.id,
    model="gpt-4o-2024-08-06",
)

print(fine_tuning_job)


FineTuningJob(id='ftjob-pyTkN0SKCF8UANPWwkEb4r29', created_at=1733358301, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-oyIQPClKOf7kZAZ0IOGnlkDL', result_files=[], seed=1131686429, status='validating_files', trained_tokens=None, training_file='file-5g5CQfb7D6vZ5vnmsnaCtc', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


In [3]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-pyTkN0SKCF8UANPWwkEb4r29', created_at=1733358301, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-oyIQPClKOf7kZAZ0IOGnlkDL', result_files=[], seed=1131686429, status='validating_files', trained_tokens=None, training_file='file-5g5CQfb7D6vZ5vnmsnaCtc', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-3bN7xAyCOwtksq2uvGxSOdAj', created_at=1733353729, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:texas-a-m::Aasy5tgO', finished_at=1733354468, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-oyIQPClKOf7kZAZ0IOGnlkDL

In [6]:
client.fine_tuning.jobs.retrieve("ftjob-pyTkN0SKCF8UANPWwkEb4r29")

# client.fine_tuning.jobs.retrieve("ftjob-ZgVDlL5wKpulxNhuFSJWbphU")

FineTuningJob(id='ftjob-pyTkN0SKCF8UANPWwkEb4r29', created_at=1733358301, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:texas-a-m::AauiLOcH', finished_at=1733361180, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-oyIQPClKOf7kZAZ0IOGnlkDL', result_files=['file-64H4Nt7ESaePe1BKX3sVDb'], seed=1131686429, status='succeeded', trained_tokens=1709187, training_file='file-5g5CQfb7D6vZ5vnmsnaCtc', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [5]:
print("key", os.getenv("OPENAI_API_KEY"))

key sk-w6Kj-Sg1ie9D1bBc3KkCCNe253TEVFvOWG87_qewCTT3BlbkFJNDqY6AsLGVc3L7XGrFrK-f9TvuOUHC3E4RmEEC3qEA


In [11]:
import tiktoken
import json

# 加载训练数据文件
fine_tuning_file = "data/gpt4_fine_tuning_train.jsonl"

# 加载 GPT-4 编码器
enc = tiktoken.encoding_for_model("gpt-4")

# 计算训练数据的总 token 数
total_tokens = 0
with open(fine_tuning_file, 'r') as f:
    for line in f:
        data = json.loads(line)
        messages = data["messages"]
        for message in messages:
            total_tokens += len(enc.encode(message["content"]))

print(f"Total tokens in training data: {total_tokens}")
training_cost_per_1M_tokens = 25.000  # 训练成本 $25.000 / 1M tokens
fine_tuning_cost = (186450 / 1_000_000) * training_cost_per_1M_tokens
print(f"Estimated fine-tuning cost: ${fine_tuning_cost:.2f}")

Total tokens in training data: 6292067
Estimated fine-tuning cost: $4.66


In [36]:

import pandas as pd

# 加载完整数据集和用于微调的数据
full_dataset_file = "data/train.csv"  
fine_tuned_file = "data/sample_train_data.csv"  

# 读取数据集
full_dataset = pd.read_csv(full_dataset_file)
fine_tuned_data = pd.read_csv(fine_tuned_file)

# 找到剩余条目
remaining_data = full_dataset[~full_dataset.index.isin(fine_tuned_data.index)]

# 只保留 vul = 1 的数据
vulnerable_data = remaining_data[remaining_data['vul'] == 0]

# 保存筛选后的数据
rag_file = "data/RAG_data_non_vul.csv"
vulnerable_data.to_csv(rag_file, index=False)

print(f"Vulnerable data saved to {rag_file}, total {len(vulnerable_data)} rows.")

# 输出一些统计信息
print(f"\nData Statistics:")
print(f"Original remaining data: {len(remaining_data)} rows")
print(f"Vulnerable data (vul=1): {len(vulnerable_data)} rows")
print(f"Percentage of vulnerable data: {(len(vulnerable_data)/len(remaining_data))*100:.2f}%")


Vulnerable data saved to data/RAG_data_non_vul.csv, total 6340 rows.

Data Statistics:
Original remaining data: 12698 rows
Vulnerable data (vul=1): 6340 rows
Percentage of vulnerable data: 49.93%


In [37]:
import pandas as pd
import json

# 加载剩余数据集
remaining_file = "data/RAG_data_non_vul.csv"
remaining_data = pd.read_csv(remaining_file)

# 初始化字典存储分组结果
grouped_data = {}

# 按 CWE ID 分组
grouped = remaining_data.groupby('CWE ID')

for cwe_id, group in grouped:
    # 每个 CWE ID 下的条目列表
    entries = []
    for _, row in group.iterrows():
        entries.append({
            "ID": row.get("ID", None),  # 替换为实际列名
            "vul": row.get("vul", None),
            "func_before": row.get("func_before", None),  # 替换为代码列名
            "CWE ID": cwe_id,
            "description": row.get("commit_message", None)
        })
    grouped_data[cwe_id] = entries

# 保存为 JSON 文件
output_file = "data/rag_data.json"
with open(output_file, "w") as f:
    json.dump(grouped_data, f, indent=4)

print(f"Grouped data saved to {output_file}, grouped by CWE IDs.")

Grouped data saved to data/rag_data.json, grouped by CWE IDs.


In [38]:
import tiktoken
import json

# 加载 JSON 文件
json_file = "data/RAG_data.json"
with open(json_file, "r") as f:
    rag_data = json.load(f)

# 加载 GPT-4 编码器
encoder = tiktoken.encoding_for_model("gpt-4")

# 计算 token 总数
total_tokens = 0

for cwe_id, entries in rag_data.items():
    for entry in entries:
        # 拼接 JSON 条目内容为文本
        text = f"""
        ID: {entry.get('ID', '')}
        Vul: {entry.get('vul', '')}
        Func_Before: {entry.get('func_before', '')}
        CWE ID: {entry.get('CWE ID', '')}
        Description: {entry.get('description', '')}
        """
        # 编码并统计 token 数
        total_tokens += len(encoder.encode(text))

print(f"Total tokens in JSON data: {total_tokens}")


Total tokens in JSON data: 2791015


In [58]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(df):
    """
    计算混淆矩阵和相关评估指标
    
    参数:
    df: 包含true_vul和pred_vul两列的DataFrame
    
    返回:
    confusion_matrix: 混淆矩阵
    metrics: 包含准确率、精确率、召回率和F1分数的字典
    """
    # 获取真实值和预测值
    y_true = df['true_vul']
    y_pred = df['pred_vul']
    
    # 计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred)
    
    # 计算各项指标
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred)
    }
    
    return cm, metrics

# 创建示例数据

df3 = pd.read_csv('validation_results_3_contexts.csv')

df5 = pd.read_csv('validation_results_5_contexts3.csv')

df10 = pd.read_csv('validation_results_10_contexts.csv')

tf = pd.read_csv('test_results_5_contexts3_3.csv')
# 计算混淆矩阵和指标
conf_matrix, metrics = calculate_metrics(tf)


# 打印结果
print("混淆矩阵:")
print(conf_matrix)
print("\n评估指标:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.3f}")

混淆矩阵:
[[632  68]
 [ 10 690]]

评估指标:
accuracy: 0.944
precision: 0.910
recall: 0.986
f1: 0.947


In [40]:
rag = pd.read_csv('data/RAG_data_non_vul.csv')

rag.head(20)

,CVE ID,CVE Page,CWE ID,codeLink,commit_id,commit_message,func_after,func_before,lang,project,vul
0,CVE-2014-0182,https://www.cvedetails.com/cve/CVE-2014-0182/,CWE-119,https://git.qemu.org/?p=qemu.git;a=commitdiff;...,a890a2f9137ac3cf5b607649e66a6f3a5512d8dc,NaN,static inline uint16_t vring_avail_flags(VirtQ...,static inline uint16_t vring_avail_flags(VirtQ...,C,qemu,0
1,CVE-2017-14604,https://www.cvedetails.com/cve/CVE-2017-14604/,CWE-20,https://github.com/GNOME/nautilus/commit/1630f...,1630f53481f445ada0a455e9979236d31a8d3bb0,mime-actions: use file metadata for trusting d...,link_info_stop (NautilusDirectory *directory)\...,link_info_stop (NautilusDirectory *directory)\...,C,nautilus,0
2,CVE-2018-12247,https://www.cvedetails.com/cve/CVE-2018-12247/,CWE-476,https://github.com/mruby/mruby/commit/55edae02...,55edae0226409de25e59922807cb09acb45731a2,Allow `Object#clone` to copy frozen status onl...,"mrb_obj_frozen(mrb_state *mrb, mrb_value self)...","mrb_obj_frozen(mrb_state *mrb, mrb_value self)...",C,mruby,0
3,CVE-2013-2921,https://www.cvedetails.com/cve/CVE-2013-2921/,CWE-399,https://github.com/chromium/chromium/commit/12...,1228817ab04a14df53b5a8446085f9c03bf6e964,repairs CopyFromCompositingSurface in HighDPI\...,void DelegatedFrameHost::OnCompositingAborted(...,void DelegatedFrameHost::OnCompositingAborted(...,C,Chrome,0
4,CVE-2019-12904,https://www.cvedetails.com/cve/CVE-2019-12904/,CWE-310,https://github.com/gpg/libgcrypt/commit/daedbb...,daedbbb5541cd8ecda1459d3b843ea4d92788762,AES: move look-up tables to .data section and ...,"do_decrypt_fn (const RIJNDAEL_context *ctx, un...","do_decrypt_fn (const RIJNDAEL_context *ctx, un...",C,libgcrypt,0
5,CVE-2017-12187,https://www.cvedetails.com/cve/CVE-2017-12187/,CWE-20,https://cgit.freedesktop.org/xorg/xserver/comm...,cad5a1050b7184d828aef9c1dd151c3ab649d37e,NaN,PanoramiXRenderTriFan(ClientPtr client)\n{\n ...,PanoramiXRenderTriFan(ClientPtr client)\n{\n ...,C,xserver,0
6,CVE-2016-7944,https://www.cvedetails.com/cve/CVE-2016-7944/,CWE-190,https://cgit.freedesktop.org/xorg/lib/libXfixe...,61c1039ee23a2d1de712843bed3480654d7ef42e,NaN,"XFixesSetGCClipRegion (Display *dpy, GC gc,\n\...","XFixesSetGCClipRegion (Display *dpy, GC gc,\n\...",C,libXfixes,0
7,CVE-2013-2856,https://www.cvedetails.com/cve/CVE-2013-2856/,CWE-416,https://github.com/chromium/chromium/commit/e6...,e68fafe04d29810cebe8d25554863b0cae4c1356,"Map posix error codes in bind better, and fix ...",virtual ~ReadDelegate() {}\n,virtual ~ReadDelegate() {}\n,C,Chrome,0
8,CVE-2017-5039,https://www.cvedetails.com/cve/CVE-2017-5039/,CWE-416,https://github.com/chromium/chromium/commit/69...,69b4b9ef7455753b12c3efe4eec71647e6fb1da1,Disable all DRP URL fetches when holdback is e...,std::string GetAndReset() {\n base::AutoL...,std::string GetAndReset() {\n base::AutoL...,C,Chrome,0
9,CVE-2015-5307,https://www.cvedetails.com/cve/CVE-2015-5307/,CWE-399,https://github.com/torvalds/linux/commit/54a20...,54a20552e1eae07aa240fa370a0293e006b5faed,KVM: x86: work around infinite loop in microco...,static int handle_interrupt_window(struct kvm_...,static int handle_interrupt_window(struct kvm_...,C,linux,0
